# TL;DR

# Introduction

In the [previous article](https://pintail.xyz/posts/post-merge-mev/) we attempted to model post-merge validator returns based on pre-merge data. Ethereum's merge finally occurred on 15 September 2022 (at a block height of 15537394 on the execution layer, and slot number 4700013 on the consensus layer). Now, with 6 weeks of post-merge data available we're going to take a look at the distribution of rewards actually earned by proof-of-stake validators, and compare this with the consensus layer rewards, and our model from the previous article.

We'll also take a look at one of the most significant changes to occur in parallel with the merge, which is the outsourcing of block building by validators. This change came about because of the adoption of Flashbots' MEV-boost middleware. Prior to the merge, miners were able to accept "bundles" of transactions exploiting MEV opportunities, and were trusted to include them without stealing the MEV they contained (the topic of MEV was covered [in more detail](https://pintail.xyz/posts/post-merge-mev/#but-whats-mev) previously). This system worked because under proof-of-work, Ethereum blocks were built by only a small number of mining pools, these pools could be punished for misbehaving by simply cutting them off from receiving future profitable transaction bundles. The situation changed with the transition to proof-of-stake. Now, blocks can be built and proposed by a wide rang of actors including solo validators. It is no longer feasible for all of them to have trusted relationships with all of them.

The change that was made was to allow outsourced builders to produce entire blocks rather than just bundles of transactions — that way builders could produce a block header and get this signed by the proposer _without_ first releasing the body of the block and risking the MEV being stolen. What had not been widely understood prior to [OFAC's designation of the contracts associated with privacy layer Tornado Cash](https://home.treasury.gov/news/press-releases/jy0916), was that by outsourcing the entirety of block building, rather than just provision of transaction bundles, the issue of transaction censorship became a [more pressing concern](https://time.com/6223034/ethereum-merge-sanctions-flashbots/).

# Dataset and Pre-processing

To obtain the data, we're Flashbots' mev-inspect-py tool as before, but with some [minor modifications](https://github.com/pintail-xyz/mev-inspect-py/tree/since-the-merge) to allow for handling of post-merge blocks. Our dataset starts at the Merge block, and goes up to block number 15837692 which was mined on 27 October 2022, for a full 6 weeks of data. To allow us to make sense of this data in the context of proof-of-stake Ethereum, we need to do a couple of pieces of pre-processing.

## Step 1: Identify Validators

Since we're obtaining all our MEV data from the execution layer, this data initially contains no notion of proof-of-stake validators, with a validator index and public key. We therefore need to obtain this information from a consensus layer client. For this article we're using the [Nimbus](https://github.com/status-im/nimbus-eth2) consensus layer client to query the node for the [blocks directly](https://ethereum.github.io/beacon-APIs/#/Beacon/getBlockV2), to enable us to link an execution layer block height with its consensus layer slot number. We'll also query for the validator indices of the [proposers of each epoch](https://ethereum.github.io/beacon-APIs/#/Validator/getProposerDuties), to identify which validator proposed (or failed to propose) a block in each slot.

## Step 2: Calculate Consensus Layer Rewards

To enable us to compare the level of rewards received on the execution and consensus layers, we need will also query the [validator balances](https://ethereum.github.io/beacon-APIs/#/Beacon/getStateValidators) for the start and of the period under consideration. The net reward for each validator is found as the difference between the initial and final balances for the period.

## Step 3: Identify Builders and Builder-payees

The final step is to identify cases where the validator has outsourced block building to a separate block builder. This is slighty less straightforward. On the execution layer, what used to be the "miner address" is now the "fee recipient". If the proposer has used a block builder, then this fee recipient will be set as the builder's address, but there is no record of the validator's own execution layer address provided — we will need to make some assuptions to determine it.

The proposing validator will need to be paid, and the most common way this is done is for the block produced by the builder to include a transaction which them the fees. In principle validator could be paid some other way (e.g. an 'out of band' or off-chain payment), but we will assume for this analysis that outsourced blocks include a fee payment to the proposer. We can identify such transactions from the mev-inspect-py database with the following SQL query:

So, we are making the assumption that if there is an outgoing transaction from the "miner address", in fact this address belongs to the block builder, and represents a fee payment being sent to the proposer.

The results of running the above query have been saved in a [CSV file](https://github.com/pintail-xyz/since-the-merge/blob/main/fee_recipient_transactions.csv).

In [ ]:
# imports and definitions

import csv
import json
import requests
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

CL_NODE_URL = 'http://localhost:5052'
SECONDS_PER_YEAR = 31556952

class Ecdf(pd.Series):
    def __init__(self, data):
        s = pd.Series(data)
        super().__init__(s.value_counts().sort_index().cumsum()*1./len(s))

    def get_quantile(self, q):
        return self.index[np.argmax(self.array >= q)]